# Group Project: Movie Recommendations (2487-T2 Machine Learning) [Group 2]
- Nova School of Business and Economics, Portugal
- Instructor: Qiwei Han, Ph.D.
- Program: Masters Program in Business Analytics
- Group Members: 
    - **Luca Silvano Carocci (53942)**
    - **Fridtjov Höyerholt Stokkeland (52922)**
    - **Diego García Rieckhof (53046)**
    - **Matilde Pesce (53258)**
    - **Florian Fritz Preiss (54385)**<br>
---

# Phase 4: Modelling [05 Modelling]

## 4.1 Modelling of Content-Based Recommender Systems

Content-based recommendation models have gained significant attention due to their ability to provide personalized and relevant suggestions to users based on the intrinsic characteristics of items. In the context of Streamify, the development of an effective content-based recommendation system is crucial to enhancing user experience and ensuring customer satisfaction. In this part, we focus on the investigation and evaluation of various text representation techniques, with the goal of identifying the most suitable approach for Streamify's business case.

The four content-based recommendation models examined in this part encompass a diverse range of text representation techniques, including the term frequency-inverse document frequency (TF-IDF) vectorizer, the count vectorizer, Word2Vec and Doc2Vec (Mikolov et al., 2013; Pennington et al., 2014). These models were selected due to their ability to capture semantic relationships between movie descriptions, with each offering a distinct approach to text representation that contributes to a comprehensive evaluation of their respective merits (Mikolov et al., 2013; Pennington et al., 2014).

To measure the similarity between movies, cosine similarity was employed as the primary similarity metric. This choice was based on the observation that cosine similarity is less sensitive to document length and focuses on the angular distance between vectors, making it a suitable option for comparing high-dimensional text representations (Manning et al., 2008). In addition, cosine similarity has been widely adopted in text analysis and information retrieval tasks, demonstrating its effectiveness in various domains (Huang, 2008).

In contrast to the previously discussed approach, for an optimized recommender system, the NearestNeighbor function was used, relying on the brute-force algorithm and cosine similarity as the similarity metric. The brute-force algorithm was chosen due to its robustness and simplicity, as it directly computes the pairwise similarities between all points in the dataset (Hastie et al., 2009).

The use of cosine similarity as the similarity metric further enhances the content-based recommender system's performance. As mentioned earlier, cosine similarity is less sensitive to document length and is adept at capturing the semantic relationships between high-dimensional text representations (Manning et al., 2008). This combination of the brute-force algorithm and cosine similarity enables the recommender system to provide more accurate and relevant recommendations to users, thereby improving the overall quality of the recommendations.

In summary, by employing the brute-force algorithm and cosine similarity, the optimized content-based recommender system can better capture semantic relationships between movie descriptions and provide more accurate recommendations, contributing to enhanced user satisfaction and a more engaging user experience.

**Rationale for the 'recommend()' function within each Recommender Class:**

The recommend function in the given code snippet plays a crucial role in the content-based recommender system. It takes as input a movie_id and an optional parameter top_n to return the top n recommended movies based on their content similarity, sentiment, and other factors.

- First, the function retrieves the index of the input movie in the movies_df DataFrame using the provided movie_id. This index is stored in the movie_index variable.

- Next, the cosine_similarity method or kneighbors method is called on the top_k_similar_movies object to obtain the cosine similarities or distances and indices (top_k_indices) of the k-nearest neighbors to the input movie. The cosine similarities or k-nearest neighbors are computed using the precomputed respective matrix for the movie descriptions.

- The first index in the top_k_indices list corresponds to the input movie itself, so it is excluded by slicing the list from the second element onwards.

- The selected movie indices are then used to subset the movies_df DataFrame, extracting relevant information such as the title, vote count, vote average, score, and sentiment for each recommended movie.

- The vote count and vote average columns in the recommendations DataFrame are converted to integers.

- The sentiment difference between the input movie and the recommended movies is calculated as the absolute difference in sentiment scores, and this value is added as a new column to the recommendations DataFrame.

- The average vote (C) and the 0.6 quantile of the vote count (m) are computed for the recommended movies.

- The qualified DataFrame is created by filtering the recommendations DataFrame to include only those movies with a vote count greater than or equal to m, and non-null values for both vote count and vote average.

-  The score, sentiment difference, and cosine similarity columns in the qualified DataFrame are scaled using the MinMaxScaler to ensure that they are on the same scale.

- A combined score is computed for each qualified movie using a weighted average of the scaled score (weight: 0.1), cosine similarity (weight: 0.7), and the inverse of the scaled sentiment difference (weight: 0.2). This approach balances the importance of various factors in generating the recommendations.

- The qualified DataFrame is sorted by the combined score in descending order, and the top n movies are returned as the final recommendations.

This approach to generating recommendations integrates both content similarity and sentiment analysis to provide a more comprehensive and accurate list of movie suggestions. The combined score ensures that movies with high content similarity, a close match in sentiment, and a good overall score are prioritized in the recommendations. By considering these factors, the recommender system can provide personalized and relevant movie suggestions that cater to individual user preferences (Ricci et al., 2011; Pazzani & Billsus, 2007).

In [1]:
# Standard library imports
import os
import sys
import time
import warnings

# Third-party imports
from collections import defaultdict
import joblib
import numpy as np
import pandas as pd
import pickle
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk import word_tokenize
from nltk.corpus import stopwords
from pympler import asizeof
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from sentence_transformers import SentenceTransformer
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM, TextDatasetForNextSentencePrediction, Trainer, TrainingArguments
import torch

warnings.filterwarnings("ignore")

In [2]:
# Load movies dataset
movies_df = pd.read_csv('../00_Data/02_engineered/movies_df_engineered.csv', dtype={'movieId': int})
movies_df.head(2)

,movieId,title,movie_age,genres,combined_text,vote_average,vote_count,score,sentiment
0,1,Toy Story (1995),28,"['Adventure', 'Animation', 'Children', 'Comedy...",adventure animation children comedy fantasy re...,3.893708,57309.0,3.883305,0.112121
1,2,Jumanji (1995),28,"['Adventure', 'Children', 'Fantasy']",adventure children fantasy adaptationofbook ad...,3.251527,24228.0,3.242912,-0.218750


In [3]:
# Load ratings dataset
ratings_df = pd.read_csv('../00_Data/01_processed/prepr_ratings.csv', dtype={'userId': object, 'movieId': int})
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'], unit='s', origin='unix')
ratings_df = ratings_df.drop('Unnamed: 0', axis=1)
ratings_df.head(2)

,userId,movieId,rating,timestamp
0,1,296,5.0,2006-05-17 15:34:04
1,1,306,3.5,2006-05-17 12:26:57


### **4.1.1 TF-IDF Vectorizer**

In [4]:
class ContentRecommenderTFIDF:

    def __init__(self, movies_df, k=100):
        self.movies_df = movies_df
        self.tfidf = self.train_tfidf()
        self.top_k_similar_movies = self.get_top_k_similar_movies(k)
        self.scaler = MinMaxScaler()

    def train_tfidf(self):
        tfidf_vector = TfidfVectorizer(stop_words='english', ngram_range=(1, 3), max_df=1305, min_df=5, sublinear_tf=True)
        tfidf_matrix = tfidf_vector.fit_transform(self.movies_df['combined_text'])
        return csr_matrix(tfidf_matrix)

    def get_top_k_similar_movies(self, k):
        similarity_matrix = cosine_similarity(self.tfidf)
        top_k_similar_movies = {}

        for i in range(similarity_matrix.shape[0]):
            top_k_indices = np.argsort(similarity_matrix[i])[::-1][1:k+1]
            top_k_similar_movies[i] = top_k_indices

        return top_k_similar_movies

    def recommend(self, movie_id, top_n=10):

        movie_index = self.movies_df[self.movies_df['movieId'] == movie_id].index[0]
        top_k_indices = self.top_k_similar_movies[movie_index]
        
        recommendations = self.movies_df.iloc[top_k_indices][['movieId', 'title', 'vote_count', 'vote_average', 'score', 'sentiment']]
        movie_similarities = cosine_similarity(self.tfidf[movie_index], self.tfidf[top_k_indices]).flatten()

        recommendations['cosine_similarity'] = movie_similarities

        recommendations['vote_count'] = recommendations['vote_count'].astype('int')
        recommendations['vote_average'] = recommendations['vote_average'].astype('int')

        input_movie_sentiment = self.movies_df.loc[movie_index, 'sentiment']
        recommendations['sentiment_difference'] = np.abs(recommendations['sentiment'] - input_movie_sentiment)

        C = recommendations['vote_average'].mean()
        m = recommendations['vote_count'].quantile(0.6)

        qualified = recommendations[(recommendations['vote_count'] >= m) & (recommendations['vote_count'].notnull()) & (recommendations['vote_average'].notnull())]

        qualified.loc[:, ['score', 'sentiment_difference', 'cosine_similarity']] = self.scaler.fit_transform(qualified[['score', 'sentiment_difference', 'cosine_similarity']])
        qualified.loc[:, 'combined_score'] = qualified['score'] * 0.1 + qualified['cosine_similarity'] * 0.7 + (1 - qualified['sentiment_difference']) * 0.2
        qualified = qualified.sort_values('combined_score', ascending=False).head(top_n)

        return qualified

In [5]:
class ContentRecommenderTFIDFOptimized:

    def __init__(self, movies_df, k=100):
        self.movies_df = movies_df
        self.tfidf = self.train_tfidf()
        self.top_k_similar_movies = self.get_top_k_similar_movies(k)
        self.scaler = MinMaxScaler()

    def train_tfidf(self):
        tfidf_vector = TfidfVectorizer(stop_words='english', ngram_range=(1, 3), max_df=1305, min_df=5, sublinear_tf=True)
        tfidf_matrix = tfidf_vector.fit_transform(self.movies_df['combined_text'])
        return csr_matrix(tfidf_matrix)

    def get_top_k_similar_movies(self, k):
        nbrs = NearestNeighbors(n_neighbors=k + 1, algorithm='brute', metric='cosine').fit(self.tfidf)
        return nbrs

    def recommend(self, movie_id, top_n=10):
        movie_index = self.movies_df[self.movies_df['movieId'] == movie_id].index[0]
        distances, top_k_indices = self.top_k_similar_movies.kneighbors(self.tfidf[movie_index])
        top_k_indices = top_k_indices[0][1:]

        recommendations = self.movies_df.iloc[top_k_indices][['title', 'vote_count', 'vote_average', 'score', 'sentiment']]
        recommendations['cosine_similarity'] = 1 - distances[0][1:]
        recommendations['vote_count'] = recommendations['vote_count'].astype('int')
        recommendations['vote_average'] = recommendations['vote_average'].astype('int')

        input_movie_sentiment = self.movies_df.loc[movie_index, 'sentiment']
        recommendations['sentiment_difference'] = np.abs(recommendations['sentiment'] - input_movie_sentiment)

        C = recommendations['vote_average'].mean()
        m = recommendations['vote_count'].quantile(0.6)

        qualified = recommendations[(recommendations['vote_count'] >= m) & (recommendations['vote_count'].notnull()) & (recommendations['vote_average'].notnull())]
        qualified.loc[:, ['score', 'sentiment_difference', 'cosine_similarity']] = self.scaler.fit_transform(qualified[['score', 'sentiment_difference', 'cosine_similarity']])
        qualified.loc[:, 'combined_score'] = qualified['score'] * 0.1 + qualified['cosine_similarity'] * 0.7 + (1 - qualified['sentiment_difference']) * 0.2
        qualified = qualified.sort_values('combined_score', ascending=False).head(top_n)

        return qualified

**Rationale for each parameter chosen for the TF-IDF Vectorizer:**

1. **ngram_range:** The ngram_range parameter is set to (1, 3) to capture unigrams, bigrams, and trigrams. This allows the model to consider not only individual words but also meaningful combinations of words that appear in your dataset. By considering these n-grams, the model can capture the semantic relationships between words and phrases in the movie descriptions, leading to a more comprehensive understanding of the content and thus enabling better recommendations.

2. **max_df:** The max_df parameter has been assigned a value of 1305 to exclude words with a document frequency exceeding the specified threshold. This approach aids in removing overly frequent words that do not contribute significantly to the quality of recommendations. As the most impactful unigrams tend to be common words such as "life" and "one," assigning a lower max_df value mitigates their influence on the recommendations. By filtering out these frequent words, the model can concentrate more effectively on meaningful terms that differentiate movies, leading to more precise and pertinent recommendations. The value of 1305 was chosen based on the exploratory data analysis, which indicated the importance of incorporating bigrams and trigrams into the model. As the highest occurring bigram had a frequency of 1303, this value was selected to encompass all bigrams while simultaneously excluding any extraneous unigrams above it.

1. **min_df:** The min_df parameter is set to 5 to exclude words that have a document frequency lower than the given threshold. This helps remove rare words that could lead to overfitting or noisy recommendations. Rare words may not generalize well to other movies, and their presence in the recommendations may introduce noise or irrelevant information. By setting a minimum document frequency threshold, the model ensures that terms are present in multiple documents, increasing their relevance and leading to more robust recommendations.

2. **sublinear_tf:** The sublinear_tf parameter is set to True to apply sublinear scaling to term frequencies (i.e., replace tf with 1 + log(tf)). This can help reduce the impact of very high term frequencies on the recommendations. In some cases, high term frequencies can disproportionately influence the recommendations, even when the term is not necessarily relevant. Using sublinear scaling reduces this impact, allowing the model to focus on more meaningful terms and relationships, resulting in improved recommendations that are less influenced by extreme term frequencies.


In summary, these parameter choices for the TfidfVectorizer help create a more comprehensive, accurate, and robust model for movie recommendations. By considering n-grams, filtering out overly common and rare words, and applying sublinear scaling, the model can better capture the important features in your movie dataset and generate more meaningful and relevant recommendations.

In [6]:
# Create an instance of the recommender class
%time recommenderTFIDF = ContentRecommenderTFIDF(movies_df)

CPU times: total: 1min 12s
Wall time: 2min 25s


In [7]:
# Create an instance of the recommender class
%time recommenderTFIDFOptimized = ContentRecommenderTFIDFOptimized(movies_df)

CPU times: total: 9.11 s
Wall time: 11 s


In [8]:
# Get recommendations for a specific movie
movie_id = 1
top_n = 10

recommendationsTFIDF = recommenderTFIDF.recommend(movie_id, top_n)

title = movies_df[movies_df['movieId'] == movie_id]['title'].to_string(index=False, header=False)
print(f"\nTop {top_n} recommendations for {title}:\n")
recommendationsTFIDF


Top 10 recommendations for Toy Story (1995):



,movieId,title,vote_count,vote_average,score,sentiment,cosine_similarity,sentiment_difference,combined_score
2877,3114,Toy Story 2 (1999),26536,3,0.635514,0.380000,1.000000,0.438035,0.875944
14080,78499,Toy Story 3 (2010),14426,3,0.652222,-0.050000,0.469961,0.262557,0.541683
4565,4886,"Monsters, Inc. (2001)",34572,3,0.660061,0.060000,0.314434,0.080040,0.470102
7884,8961,"Incredibles, The (2004)",30562,3,0.662469,0.233333,0.245444,0.194679,0.399122
2145,2355,"Bug's Life, A (1998)",22471,3,0.492689,0.500000,0.371675,0.637145,0.382013
10349,45517,Cars (2006),8147,3,0.340392,-0.037500,0.280130,0.241816,0.381767
5976,6377,Finding Nemo (2003),34712,3,0.651518,0.325000,0.204601,0.346776,0.339017
4007,4306,Shrek (2001),42303,3,0.606683,0.155556,0.125416,0.065626,0.335334
1176,1270,Back to the Future (1985),49595,3,0.724980,0.062500,0.090035,0.075892,0.320344
27314,134853,Inside Out (2015),13580,3,0.693456,0.162500,0.089328,0.077149,0.316446


In [9]:
# Get recommendations for a specific movie
movie_id = 1
top_n = 10

recommendationsTFIDFOptimized = recommenderTFIDFOptimized.recommend(movie_id, top_n)

title = movies_df[movies_df['movieId'] == movie_id]['title'].to_string(index=False, header=False)
print(f"\nTop {top_n} recommendations for {title}:\n")
recommendationsTFIDFOptimized


Top 10 recommendations for Toy Story (1995):



,title,vote_count,vote_average,score,sentiment,cosine_similarity,sentiment_difference,combined_score
2877,Toy Story 2 (1999),26536,3,0.635514,0.380000,1.000000,0.438035,0.875944
14080,Toy Story 3 (2010),14426,3,0.652222,-0.050000,0.469961,0.262557,0.541683
4565,"Monsters, Inc. (2001)",34572,3,0.660061,0.060000,0.314434,0.080040,0.470102
7884,"Incredibles, The (2004)",30562,3,0.662469,0.233333,0.245444,0.194679,0.399122
2145,"Bug's Life, A (1998)",22471,3,0.492689,0.500000,0.371675,0.637145,0.382013
10349,Cars (2006),8147,3,0.340392,-0.037500,0.280130,0.241816,0.381767
5976,Finding Nemo (2003),34712,3,0.651518,0.325000,0.204601,0.346776,0.339017
4007,Shrek (2001),42303,3,0.606683,0.155556,0.125416,0.065626,0.335334
1176,Back to the Future (1985),49595,3,0.724980,0.062500,0.090035,0.075892,0.320344
27314,Inside Out (2015),13580,3,0.693456,0.162500,0.089328,0.077149,0.316446


In [10]:
# Save the TF-IDF Vectorizer recommender model
start_time = time.time()

with open('../02_Models/content_recommender_tfidf.pkl', 'wb') as file:
    pickle.dump(recommenderTFIDFOptimized, file)

end_time = time.time()

tfidf_save_time = end_time - start_time

print(f"Time taken to save TF-IDF Vectorizer model: {tfidf_save_time:.2f} seconds")

Time taken to save TF-IDF Vectorizer model: 0.24 seconds


In [11]:
# Measure the size of the recommender object
size_in_bytes = asizeof.asizeof(recommenderTFIDFOptimized)
size_in_kb = size_in_bytes / 1024
size_in_mb = size_in_kb / 1024

print(f"The size of the recommender object is approximately {size_in_bytes} bytes, {size_in_kb:.2f} KB, or {size_in_mb:.2f} MB.")

The size of the recommender object is approximately 120798112 bytes, 117966.91 KB, or 115.20 MB.


### **4.1.2 Count Vectorizer**

In [12]:
class ContentRecommenderCountVec:
    def __init__(self, movies_df, k=100):
        self.movies_df = movies_df
        self.count_vec = self.train_count_vec()
        self.top_k_similar_movies = self.get_top_k_similar_movies(k)
        self.scaler = MinMaxScaler()

    def train_count_vec(self):
        count_vector = CountVectorizer(stop_words='english', ngram_range=(1, 3), max_df=1305, min_df=5)
        count_matrix = count_vector.fit_transform(self.movies_df['combined_text'])
        return csr_matrix(count_matrix)

    def get_top_k_similar_movies(self, k):
        similarity_matrix = cosine_similarity(self.count_vec)
        top_k_similar_movies = {}

        for i in range(similarity_matrix.shape[0]):
            top_k_indices = np.argsort(similarity_matrix[i])[::-1][1:k+1]
            top_k_similar_movies[i] = top_k_indices

        return top_k_similar_movies

    def recommend(self, movie_id, top_n=10):
        movie_index = self.movies_df[self.movies_df['movieId'] == movie_id].index[0]
        top_k_indices = self.top_k_similar_movies[movie_index]

        recommendations = self.movies_df.iloc[top_k_indices][['title', 'vote_count', 'vote_average', 'score', 'sentiment']]
        movie_similarities = cosine_similarity(self.count_vec[movie_index], self.count_vec[top_k_indices]).flatten()

        recommendations['cosine_similarity'] = movie_similarities

        recommendations['vote_count'] = recommendations['vote_count'].astype('int')
        recommendations['vote_average'] = recommendations['vote_average'].astype('int')

        input_movie_sentiment = self.movies_df.loc[movie_index, 'sentiment']
        recommendations['sentiment_difference'] = np.abs(recommendations['sentiment'] - input_movie_sentiment)

        C = recommendations['vote_average'].mean()
        m = recommendations['vote_count'].quantile(0.6)

        qualified = recommendations[(recommendations['vote_count'] >= m) & (recommendations['vote_count'].notnull()) & (recommendations['vote_average'].notnull())]

        qualified.loc[:, ['score', 'sentiment_difference', 'cosine_similarity']] = self.scaler.fit_transform(qualified[['score', 'sentiment_difference', 'cosine_similarity']])
        qualified.loc[:, 'combined_score'] = qualified['score'] * 0.1 + qualified['cosine_similarity'] * 0.7 + (1 - qualified['sentiment_difference']) * 0.2
        qualified = qualified.sort_values('combined_score', ascending=False).head(top_n)

        return qualified

In [13]:
class ContentRecommenderCountVecOptimized:
    def __init__(self, movies_df, k=100):
        self.movies_df = movies_df
        self.count_vector = self.train_count_vector()
        self.top_k_similar_movies = self.get_top_k_similar_movies(k)
        self.scaler = MinMaxScaler()

    def train_count_vector(self):
        count_vector = CountVectorizer(stop_words='english', ngram_range=(1, 3), max_df=1305, min_df=5)
        count_matrix = count_vector.fit_transform(self.movies_df['combined_text'])
        return csr_matrix(count_matrix)

    def get_top_k_similar_movies(self, k):
        nbrs = NearestNeighbors(n_neighbors=k + 1, algorithm='brute', metric='cosine').fit(self.count_vector)
        return nbrs

    def recommend(self, movie_id, top_n=10):
        movie_index = self.movies_df[self.movies_df['movieId'] == movie_id].index[0]
        distances, top_k_indices = self.top_k_similar_movies.kneighbors(self.count_vector[movie_index])
        top_k_indices = top_k_indices[0][1:]

        recommendations = self.movies_df.iloc[top_k_indices][['title', 'vote_count', 'vote_average', 'score', 'sentiment']]

        recommendations['cosine_similarity'] = 1 - distances[0][1:]

        recommendations['vote_count'] = recommendations['vote_count'].astype('int')
        recommendations['vote_average'] = recommendations['vote_average'].astype('int')

        input_movie_sentiment = self.movies_df.loc[movie_index, 'sentiment']
        recommendations['sentiment_difference'] = np.abs(recommendations['sentiment'] - input_movie_sentiment)

        C = recommendations['vote_average'].mean()
        m = recommendations['vote_count'].quantile(0.6)

        qualified = recommendations[(recommendations['vote_count'] >= m) & (recommendations['vote_count'].notnull()) & (recommendations['vote_average'].notnull())]

        qualified.loc[:, ['score', 'sentiment_difference', 'cosine_similarity']] = self.scaler.fit_transform(qualified[['score', 'sentiment_difference', 'cosine_similarity']])
        qualified.loc[:, 'combined_score'] = qualified['score'] * 0.1 + qualified['cosine_similarity'] * 0.7 + (1 - qualified['sentiment_difference']) * 0.2
        qualified = qualified.sort_values('combined_score', ascending=False).head(top_n)

        return qualified

**Rationale for each parameter choice for the CountVectorizer:**

- **ngram_range:** The ngram_range parameter is set to (1, 3) to capture unigrams, bigrams, and trigrams. This allows the model to consider not only individual words but also meaningful combinations of words that appear in your dataset. By considering these n-grams, the model can capture the semantic relationships between words and phrases in the movie descriptions, leading to a more comprehensive understanding of the content and thus enabling better recommendations.

- **stop_words:** The stop_words parameter is set to 'english' to remove common English stop words from the text. Stop words usually do not carry significant meaning and can be safely removed from the text to reduce the feature space and improve the efficiency of the model. By filtering out stop words, the model can better focus on the meaningful terms that contribute to differentiating movies, leading to more accurate and relevant recommendations.

- **max_df:** The max_df parameter has been assigned a value of 1305 to exclude words with a document frequency exceeding the specified threshold. This approach aids in removing overly frequent words that do not contribute significantly to the quality of recommendations. As the most impactful unigrams tend to be common words such as "life" and "one," assigning a lower max_df value mitigates their influence on the recommendations. By filtering out these frequent words, the model can concentrate more effectively on meaningful terms that differentiate movies, leading to more precise and pertinent recommendations. The value of 1305 was chosen based on the exploratory data analysis, which indicated the importance of incorporating bigrams and trigrams into the model. As the highest occurring bigram had a frequency of 1303, this value was selected to encompass all bigrams while simultaneously excluding any extraneous unigrams above it.

- **min_df:** The min_df parameter is set to 5 to exclude words that have a document frequency lower than the given threshold. This helps remove rare words that could lead to overfitting or noisy recommendations. Rare words may not generalize well to other movies, and their presence in the recommendations may introduce noise or irrelevant information. By setting a minimum document frequency threshold, the model ensures that terms are present in multiple documents, increasing their relevance and leading to more robust recommendations.

In summary, these parameter choices for the CountVectorizer help create a more comprehensive, accurate, and robust model for movie recommendations. By considering n-grams, filtering out stop words, and adjusting document frequency thresholds, the model can better capture the important features in your movie dataset and generate more meaningful and relevant recommendations. This approach adheres to the principles of professionalism, academic rigor, and impersonality, ensuring a well-structured and well-written rationale for the chosen parameter settings.

In [14]:
# Create an instance of the recommender class
%time recommenderCountVec = ContentRecommenderCountVec(movies_df)

CPU times: total: 1min 2s
Wall time: 3min 51s


In [15]:
# Create an instance of the recommender class
%time recommenderCountVecOptimized = ContentRecommenderCountVecOptimized(movies_df)

CPU times: total: 5.98 s
Wall time: 12.5 s


In [16]:
# Get recommendations for a specific movie
movie_id = 1
top_n = 10

recommendationsCountVec = recommenderCountVec.recommend(movie_id, top_n)

title = movies_df[movies_df['movieId'] == movie_id]['title'].to_string(index=False, header=False)
print(f"\nTop {top_n} recommendations for {title}:\n")
recommendationsCountVec


Top 10 recommendations for Toy Story (1995):



,title,vote_count,vote_average,score,sentiment,cosine_similarity,sentiment_difference,combined_score
2877,Toy Story 2 (1999),26536,3,0.769856,0.380000,1.000000,0.438035,0.889379
14080,Toy Story 3 (2010),14426,3,0.790096,-0.050000,0.545273,0.262557,0.608189
4565,"Monsters, Inc. (2001)",34572,3,0.799592,0.060000,0.306209,0.080040,0.478298
7884,"Incredibles, The (2004)",30562,3,0.802510,0.233333,0.237741,0.194679,0.407734
4007,Shrek (2001),42303,3,0.734931,0.155556,0.152336,0.065626,0.367003
2145,"Bug's Life, A (1998)",22471,3,0.596839,0.500000,0.306467,0.637145,0.346782
5976,Finding Nemo (2003),34712,3,0.789244,0.325000,0.177835,0.346776,0.334053
27314,Inside Out (2015),13580,3,0.840047,0.162500,0.090087,0.077149,0.331636
1894,101 Dalmatians (One Hundred and One Dalmatians...,8409,3,0.487823,0.095238,0.118205,0.021571,0.327211
10349,Cars (2006),8147,3,0.412348,-0.037500,0.182913,0.241816,0.320911


In [17]:
# Get recommendations for a specific movie
movie_id = 1
top_n = 10

recommendationsCountVecOptimized = recommenderCountVecOptimized.recommend(movie_id, top_n)

title = movies_df[movies_df['movieId'] == movie_id]['title'].to_string(index=False, header=False)
print(f"\nTop {top_n} recommendations for {title}:\n")
recommendationsCountVecOptimized


Top 10 recommendations for Toy Story (1995):



,title,vote_count,vote_average,score,sentiment,cosine_similarity,sentiment_difference,combined_score
2877,Toy Story 2 (1999),26536,3,0.769856,0.380000,1.000000,0.438035,0.889379
14080,Toy Story 3 (2010),14426,3,0.790096,-0.050000,0.545273,0.262557,0.608189
4565,"Monsters, Inc. (2001)",34572,3,0.799592,0.060000,0.306209,0.080040,0.478298
7884,"Incredibles, The (2004)",30562,3,0.802510,0.233333,0.237741,0.194679,0.407734
4007,Shrek (2001),42303,3,0.734931,0.155556,0.152336,0.065626,0.367003
2145,"Bug's Life, A (1998)",22471,3,0.596839,0.500000,0.306467,0.637145,0.346782
5976,Finding Nemo (2003),34712,3,0.789244,0.325000,0.177835,0.346776,0.334053
27314,Inside Out (2015),13580,3,0.840047,0.162500,0.090087,0.077149,0.331636
1894,101 Dalmatians (One Hundred and One Dalmatians...,8409,3,0.487823,0.095238,0.118205,0.021571,0.327211
10349,Cars (2006),8147,3,0.412348,-0.037500,0.182913,0.241816,0.320911


In [18]:
# Save the CountVectorizer recommender model
start_time = time.time()

with open('../02_Models/content_recommender_countvec.pkl', 'wb') as file:
    pickle.dump(recommenderCountVec, file)

end_time = time.time()

cv_save_time = end_time - start_time

print(f"Time taken to save CountVectorizer model: {cv_save_time:.2f} seconds")

Time taken to save CountVectorizer model: 5.88 seconds


In [19]:
# Measure the size of the recommender object
size_in_bytes = asizeof.asizeof(recommenderCountVec)
size_in_kb = size_in_bytes / 1024
size_in_mb = size_in_kb / 1024

print(f"The size of the recommender object is approximately {size_in_bytes} bytes, {size_in_kb:.2f} KB, or {size_in_mb:.2f} MB.")

The size of the recommender object is approximately 18896755000 bytes, 18453862.30 KB, or 18021.35 MB.


In [20]:
# Save the CountVectorizerOptimized recommender model
start_time = time.time()

with open('../02_Models/content_recommender_countvec_opt.pkl', 'wb') as file:
    pickle.dump(recommenderCountVecOptimized, file)

end_time = time.time()

cv_save_time = end_time - start_time

print(f"Time taken to save CountVectorizerOptimized model: {cv_save_time:.2f} seconds")

Time taken to save CountVectorizerOptimized model: 0.38 seconds


In [21]:
# Measure the size of the recommender object
size_in_bytes = asizeof.asizeof(recommenderCountVecOptimized)
size_in_kb = size_in_bytes / 1024
size_in_mb = size_in_kb / 1024

print(f"The size of the recommender object is approximately {size_in_bytes} bytes, {size_in_kb:.2f} KB, or {size_in_mb:.2f} MB.")

The size of the recommender object is approximately 120798392 bytes, 117967.18 KB, or 115.20 MB.


### **4.1.3 Word2Vec**

In [22]:
class ContentRecommenderW2V:
    def __init__(self, movies_df, k=100):
        self.movies_df = movies_df
        self.preprocessed_text = self.movies_df['combined_text'].apply(self.preprocess_text).tolist()
        self.word2vec_model = self.train_word2vec_model()
        self.movies_embeddings = self.get_movie_embeddings()
        self.top_k_similar_movies = self.get_top_k_similar_movies(k)
        self.scaler = MinMaxScaler()

    def preprocess_text(self, text):
        tokens = word_tokenize(text.lower())
        stop_words = set(stopwords.words('english'))
        return [word for word in tokens if word.isalpha() and word not in stop_words]

    def train_word2vec_model(self):
        model = Word2Vec(self.preprocessed_text, vector_size=300, window=5, min_count=3, workers=4, sg=1, epochs=10)
        return model

    def get_movie_embeddings(self):
        movie_embeddings = []
        for text in self.preprocessed_text:
            embeddings = np.mean([self.word2vec_model.wv[word] for word in text if word in self.word2vec_model.wv], axis=0)
            movie_embeddings.append(embeddings)
        return np.vstack(movie_embeddings)

    def get_top_k_similar_movies(self, k):
        similarity_matrix = cosine_similarity(self.movies_embeddings)
        top_k_similar_movies = {}

        for i in range(similarity_matrix.shape[0]):
            top_k_indices = np.argsort(similarity_matrix[i])[::-1][1:k+1]
            top_k_similar_movies[i] = top_k_indices

        return top_k_similar_movies

    def recommend(self, movie_id, top_n=10):
        movie_index = self.movies_df[self.movies_df['movieId'] == movie_id].index[0]
        top_k_indices = self.top_k_similar_movies[movie_index]

        recommendations = self.movies_df.iloc[top_k_indices][['title', 'vote_count', 'vote_average', 'score', 'sentiment']]
        movie_similarities = [cosine_similarity(self.movies_embeddings[movie_index].reshape(1, -1), self.movies_embeddings[idx].reshape(1, -1)).flatten()[0] for idx in top_k_indices]

        recommendations['cosine_similarity'] = movie_similarities

        recommendations['vote_count'] = recommendations['vote_count'].astype('int')
        recommendations['vote_average'] = recommendations['vote_average'].astype('int')

        input_movie_sentiment = self.movies_df.loc[movie_index, 'sentiment']
        recommendations['sentiment_difference'] = np.abs(recommendations['sentiment'] - input_movie_sentiment)

        C = recommendations['vote_average'].mean()
        m = recommendations['vote_count'].quantile(0.6)

        qualified = recommendations[(recommendations['vote_count'] >= m) & (recommendations['vote_count'].notnull()) & (recommendations['vote_average'].notnull())]

        qualified.loc[:, ['score', 'sentiment_difference', 'cosine_similarity']] = self.scaler.fit_transform(qualified[['score', 'sentiment_difference', 'cosine_similarity']])
        qualified.loc[:, 'combined_score'] = qualified['score'] * 0.1 + qualified['cosine_similarity'] * 0.7 + (1 - qualified['sentiment_difference']) * 0.2
        qualified = qualified.sort_values('combined_score', ascending=False).head(top_n)

        return qualified

In [23]:
class ContentRecommenderW2VOptimized:
    def __init__(self, movies_df, k=100):
        self.movies_df = movies_df
        self.wv_model = self.train_word2vec()
        self.top_k_similar_movies = self.get_top_k_similar_movies(k)
        self.scaler = MinMaxScaler()

    def tokenize(self, text):
        tokens = word_tokenize(text)
        tokens = [t.lower() for t in tokens if t.isalpha()]
        return tokens

    def train_word2vec(self):
        stop_words = set(stopwords.words("english"))
        sentences = self.movies_df['combined_text'].apply(lambda x: [word for word in self.tokenize(x) if word not in stop_words])
        wv_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)
        return wv_model

    def get_top_k_similar_movies(self, k):
        movie_embeddings = np.array([self.get_movie_embedding(movie) for movie in self.movies_df['combined_text']])
        nbrs = NearestNeighbors(n_neighbors=k + 1, algorithm='brute', metric='cosine').fit(movie_embeddings)
        return nbrs

    def get_movie_embedding(self, text):
        words = self.tokenize(text)
        words = [word for word in words if word in self.wv_model.wv]
        if len(words) == 0:
            return np.zeros(self.wv_model.vector_size)
        return np.mean(self.wv_model.wv[words], axis=0)

    def recommend(self, movie_id, top_n=10):
        movie_index = self.movies_df[self.movies_df['movieId'] == movie_id].index[0]
        movie_embedding = self.get_movie_embedding(self.movies_df.loc[movie_index, 'combined_text'])
        distances, top_k_indices = self.top_k_similar_movies.kneighbors([movie_embedding])
        top_k_indices = top_k_indices[0][1:]

        recommendations = self.movies_df.iloc[top_k_indices][['title', 'vote_count', 'vote_average', 'score', 'sentiment']]

        recommendations['cosine_similarity'] = 1 - distances[0][1:]

        recommendations['vote_count'] = recommendations['vote_count'].astype('int')
        recommendations['vote_average'] = recommendations['vote_average'].astype('int')

        input_movie_sentiment = self.movies_df.loc[movie_index, 'sentiment']
        recommendations['sentiment_difference'] = np.abs(recommendations['sentiment'] - input_movie_sentiment)

        C = recommendations['vote_average'].mean()
        m = recommendations['vote_count'].quantile(0.6)

        qualified = recommendations[(recommendations['vote_count'] >= m) & (recommendations['vote_count'].notnull()) & (recommendations['vote_average'].notnull())]
        qualified.loc[:, ['score', 'sentiment_difference', 'cosine_similarity']] = self.scaler.fit_transform(qualified[['score', 'sentiment_difference', 'cosine_similarity']])
        qualified.loc[:, 'combined_score'] = qualified['score'] * 0.1 + qualified['cosine_similarity'] * 0.7 + (1 - qualified['sentiment_difference']) * 0.2
        qualified = qualified.sort_values('combined_score', ascending=False).head(top_n)
        
        return qualified

**Rationale for each parameter choice for Word2Vec:**

- **vector_size:** A vector size of 300 is widely used in various NLP tasks and provides a good balance between capturing semantic information and computational complexity.

- **window:** A window size of 5 is suitable for capturing both local syntactic and more global semantic relationships present in the movie industry domain.

- **min_count:** A min_count of 3 strikes a balance between including meaningful words and filtering out rare, potentially noisy words.

- **workers:** Using multiple worker threads allows for efficient parallelization and speeds up the training process.

- **sg:** The skip-gram algorithm performs better on semantic tasks and with rare words, making it suitable for the movie industry domain.

- **epochs:** 10 epochs provide a balance between model performance and training time.

In summary, these parameter choices for Word2Vec help create a more comprehensive, accurate, and robust model for movie recommendations. By configuring the Word2Vec model with the optimal parameters, the model can better capture the important features in your movie dataset and generate more meaningful and relevant recommendations. This approach adheres to the principles of professionalism, academic rigor, and impersonality, ensuring a well-structured and well-written rationale for the chosen parameter settings.

In [24]:
# Create an instance of the recommender class
%time recommenderW2V = ContentRecommenderW2V(movies_df)

CPU times: total: 10min 24s
Wall time: 6min 53s


In [25]:
# Create an instance of the recommender class
%time recommenderW2VOptimized = ContentRecommenderW2VOptimized(movies_df)

CPU times: total: 39.9 s
Wall time: 39 s


In [26]:
# Get recommendations for a specific movie
movie_id = 1
top_n = 10

recommendationsW2V = recommenderW2V.recommend(movie_id, top_n)

title = movies_df[movies_df['movieId'] == movie_id]['title'].to_string(index=False, header=False)
print(f"\nTop {top_n} recommendations for {title}:\n")
recommendationsW2V


Top 10 recommendations for Toy Story (1995):



,title,vote_count,vote_average,score,sentiment,cosine_similarity,sentiment_difference,combined_score
2877,Toy Story 2 (1999),26536,3,0.769856,0.380000,1.000000,0.438035,0.889379
4565,"Monsters, Inc. (2001)",34572,3,0.799592,0.060000,0.799140,0.080040,0.823349
7884,"Incredibles, The (2004)",30562,3,0.802510,0.233333,0.773587,0.194679,0.782826
14080,Toy Story 3 (2010),14426,3,0.790096,-0.050000,0.784255,0.262557,0.775477
4007,Shrek (2001),42303,3,0.734931,0.155556,0.599053,0.065626,0.679705
3480,Chicken Run (2000),18762,3,0.538146,0.324479,0.696206,0.345912,0.671976
5976,Finding Nemo (2003),34712,3,0.789244,0.325000,0.631834,0.346776,0.651853
350,"Lion King, The (1994)",42745,3,0.779396,-0.250000,0.564211,0.594406,0.554006
12737,Up (2009),25127,3,0.876342,0.034091,0.399848,0.123029,0.542922
2145,"Bug's Life, A (1998)",22471,3,0.596839,0.500000,0.576047,0.637145,0.535488


In [27]:
# Get recommendations for a specific movie
movie_id = 1
top_n = 10

recommendationsW2VOptimized = recommenderW2VOptimized.recommend(movie_id, top_n)

title = movies_df[movies_df['movieId'] == movie_id]['title'].to_string(index=False, header=False)
print(f"\nTop {top_n} recommendations for {title}:\n")
recommendationsW2VOptimized


Top 10 recommendations for Toy Story (1995):



,title,vote_count,vote_average,score,sentiment,cosine_similarity,sentiment_difference,combined_score
4565,"Monsters, Inc. (2001)",34572,3,0.799592,0.060000,0.972687,0.094913,0.941857
2877,Toy Story 2 (1999),26536,3,0.769856,0.380000,1.000000,0.519434,0.873099
7884,"Incredibles, The (2004)",30562,3,0.802510,0.233333,0.826052,0.230855,0.812316
5976,Finding Nemo (2003),34712,3,0.789244,0.325000,0.830796,0.411217,0.778238
535,"Nightmare Before Christmas, The (1993)",21940,3,0.715405,-0.105000,0.727987,0.419564,0.697218
1064,Wallace & Gromit: The Wrong Trousers (1993),15270,4,0.977101,-0.133333,0.689040,0.475312,0.684975
2145,"Bug's Life, A (1998)",22471,3,0.596839,0.500000,0.816878,0.755543,0.680390
576,Pinocchio (1940),12742,3,0.527873,0.147222,0.578521,0.061425,0.645467
350,"Lion King, The (1994)",42745,3,0.779396,-0.250000,0.717096,0.704863,0.638934
14080,Toy Story 3 (2010),14426,3,0.790096,-0.050000,0.592281,0.311347,0.631337


In [28]:
# Save the Word2Vec recommender model
start_time = time.time()

with open('../02_Models/content_recommender_w2v.pkl', 'wb') as recommender_file:
    pickle.dump(recommenderW2V, recommender_file)
    
end_time = time.time()

word2vec_save_time = end_time - start_time
print(f"Time taken to save Word2Vec model: {word2vec_save_time:.2f} seconds")

Time taken to save Word2Vec model: 4.85 seconds


In [29]:
# Measure the size of the recommender object
size_in_bytes = asizeof.asizeof(recommenderW2V)
size_in_kb = size_in_bytes / 1024
size_in_mb = size_in_kb / 1024

print(f"The size of the recommender object is approximately {size_in_bytes} bytes, {size_in_kb:.2f} KB, or {size_in_mb:.2f} MB.")

The size of the recommender object is approximately 19218868992 bytes, 18768426.75 KB, or 18328.54 MB.


In [30]:
# Save the Word2Vec recommender model
start_time = time.time()

with open('../02_Models/content_recommender_w2v_opt.pkl', 'wb') as recommender_file:
    pickle.dump(recommenderW2VOptimized, recommender_file)
    
end_time = time.time()

word2vec_save_time = end_time - start_time
print(f"Time taken to save Word2Vec Optimized model: {word2vec_save_time:.2f} seconds")

Time taken to save Word2Vec Optimized model: 0.40 seconds


In [31]:
# Measure the size of the recommender object
size_in_bytes = asizeof.asizeof(recommenderW2VOptimized)
size_in_kb = size_in_bytes / 1024
size_in_mb = size_in_kb / 1024

print(f"The size of the recommender object is approximately {size_in_bytes} bytes, {size_in_kb:.2f} KB, or {size_in_mb:.2f} MB.")

The size of the recommender object is approximately 131367024 bytes, 128288.11 KB, or 125.28 MB.


### **4.1.4 Doc2Vec**

In [32]:
class ContentRecommenderD2V:
    def __init__(self, movies_df, k=100):
        self.movies_df = movies_df
        self.preprocessed_text = self.movies_df['combined_text'].apply(self.preprocess_text).tolist()
        self.doc2vec_model = self.train_doc2vec_model()
        self.movies_embeddings = self.get_movie_embeddings()
        self.top_k_similar_movies = self.get_top_k_similar_movies(k)
        self.scaler = MinMaxScaler()

    def preprocess_text(self, text):
        tokens = word_tokenize(text.lower())
        stop_words = set(stopwords.words('english'))
        return [word for word in tokens if word.isalpha() and word not in stop_words]

    def train_doc2vec_model(self):
        tagged_documents = [TaggedDocument(words=text, tags=[str(i)]) for i, text in enumerate(self.preprocessed_text)]
        model = Doc2Vec(tagged_documents, vector_size=300, window=5, min_count=3, workers=4, epochs=10)
        return model

    def get_movie_embeddings(self):
        movie_embeddings = [self.doc2vec_model.dv[str(i)] for i in range(len(self.preprocessed_text))]
        return np.vstack(movie_embeddings)

    def get_top_k_similar_movies(self, k):
        similarity_matrix = cosine_similarity(self.movies_embeddings)
        top_k_similar_movies = {}

        for i in range(similarity_matrix.shape[0]):
            top_k_indices = np.argsort(similarity_matrix[i])[::-1][1:k+1]
            top_k_similar_movies[i] = top_k_indices

        return top_k_similar_movies

    def recommend(self, movie_id, top_n=10):
        movie_index = self.movies_df[self.movies_df['movieId'] == movie_id].index[0]
        top_k_indices = self.top_k_similar_movies[movie_index]

        recommendations = self.movies_df.iloc[top_k_indices][['title', 'vote_count', 'vote_average', 'score', 'sentiment']]
        movie_similarities = [cosine_similarity(self.movies_embeddings[movie_index].reshape(1, -1), self.movies_embeddings[idx].reshape(1, -1)).flatten()[0] for idx in top_k_indices]

        recommendations['cosine_similarity'] = movie_similarities

        recommendations['vote_count'] = recommendations['vote_count'].astype('int')
        recommendations['vote_average'] = recommendations['vote_average'].astype('int')

        input_movie_sentiment = self.movies_df.loc[movie_index, 'sentiment']
        recommendations['sentiment_difference'] = np.abs(recommendations['sentiment'] - input_movie_sentiment)

        C = recommendations['vote_average'].mean()
        m = recommendations['vote_count'].quantile(0.6)

        qualified = recommendations[(recommendations['vote_count'] >= m) & (recommendations['vote_count'].notnull()) & (recommendations['vote_average'].notnull())]

        qualified.loc[:, ['score', 'sentiment_difference', 'cosine_similarity']] = self.scaler.fit_transform(qualified[['score', 'sentiment_difference', 'cosine_similarity']])
        qualified.loc[:, 'combined_score'] = qualified['score'] * 0.1 + qualified['cosine_similarity'] * 0.7 + (1 - qualified['sentiment_difference']) * 0.2
        qualified = qualified.sort_values('combined_score', ascending=False).head(top_n)

        return qualified

In [33]:
class ContentRecommenderD2VOptimized:
    def __init__(self, movies_df, k=100):
        self.movies_df = movies_df
        self.dv_model = self.train_doc2vec()
        self.top_k_similar_movies = self.get_top_k_similar_movies(k)
        self.scaler = MinMaxScaler()

    def tokenize(self, text):
        tokens = word_tokenize(text)
        tokens = [t.lower() for t in tokens if t.isalpha()]
        return tokens

    def train_doc2vec(self):
        stop_words = set(stopwords.words("english"))
        tagged_documents = [
            TaggedDocument(
                words=[word for word in self.tokenize(text) if word not in stop_words],
                tags=[str(index)]
            )
            for index, text in self.movies_df['combined_text'].iteritems()
        ]
        dv_model = Doc2Vec(tagged_documents, vector_size=100, window=5, min_count=5, workers=4)
        return dv_model

    def get_top_k_similar_movies(self, k):
        movie_embeddings = np.array([self.get_movie_embedding(index) for index in self.movies_df.index])
        nbrs = NearestNeighbors(n_neighbors=k + 1, algorithm='brute', metric='cosine').fit(movie_embeddings)
        return nbrs

    def get_movie_embedding(self, index):
        return self.dv_model.dv[str(index)]

    def recommend(self, movie_id, top_n=10):
        movie_index = self.movies_df[self.movies_df['movieId'] == movie_id].index[0]
        movie_embedding = self.get_movie_embedding(movie_index)
        distances, top_k_indices = self.top_k_similar_movies.kneighbors([movie_embedding])
        top_k_indices = top_k_indices[0][1:]

        recommendations = self.movies_df.iloc[top_k_indices][['title', 'vote_count', 'vote_average', 'score', 'sentiment']]

        recommendations['cosine_similarity'] = 1 - distances[0][1:]

        recommendations['vote_count'] = recommendations['vote_count'].astype('int')
        recommendations['vote_average'] = recommendations['vote_average'].astype('int')

        input_movie_sentiment = self.movies_df.loc[movie_index, 'sentiment']
        recommendations['sentiment_difference'] = np.abs(recommendations['sentiment'] - input_movie_sentiment)

        C = recommendations['vote_average'].mean()
        m = recommendations['vote_count'].quantile(0.6)

        qualified = recommendations[(recommendations['vote_count'] >= m) & (recommendations['vote_count'].notnull()) & (recommendations['vote_average'].notnull())]
        qualified.loc[:, ['score', 'sentiment_difference', 'cosine_similarity']] = self.scaler.fit_transform(qualified[['score', 'sentiment_difference', 'cosine_similarity']])
        qualified.loc[:, 'combined_score'] = qualified['score'] * 0.1 + qualified['cosine_similarity'] * 0.7 + (1 - qualified['sentiment_difference']) * 0.2
        qualified = qualified.sort_values('combined_score', ascending=False).head(top_n)
        
        return qualified

**Rationale for each parameter choice for Doc2Vec:**

- **vector_size:**  A vector size of 300 is a widely used dimensionality for various NLP tasks and provides a good balance between capturing semantic information and computational complexity.

- **window:** A window size of 5 is suitable for capturing both local syntactic and more global semantic relationships in the movie industry domain, similar to the rationale for Word2Vec.

- **min_count:** A min_count of 3 strikes a balance between including meaningful words and filtering out rare, potentially noisy words. This helps prevent overfitting and maintain computational efficiency.

- **workers:** Using multiple worker threads allows for efficient parallelization and speeds up the training process.

- **epochs:** 10 epochs provide a balance between model performance and training time.

In summary, these parameter choices for Word2Vec help create a more comprehensive, accurate, and robust model for movie recommendations. By configuring the Word2Vec model with the optimal parameters, the model can better capture the important features in your movie dataset and generate more meaningful and relevant recommendations. This approach adheres to the principles of professionalism, academic rigor, and impersonality, ensuring a well-structured and well-written rationale for the chosen parameter settings.

In [34]:
# Create an instance of the recommender class
%time recommenderD2V = ContentRecommenderD2V(movies_df)

CPU times: total: 4min 29s
Wall time: 5min 50s


In [35]:
# Create an instance of the recommender class
%time recommenderD2VOptimized = ContentRecommenderD2VOptimized(movies_df)

CPU times: total: 10.4 s
Wall time: 1min 10s


In [36]:
# Get recommendations for a specific movie
movie_id = 1
top_n = 10

recommendationsD2V = recommenderD2V.recommend(movie_id, top_n)

title = movies_df[movies_df['movieId'] == movie_id]['title'].to_string(index=False, header=False)
print(f"\nTop {top_n} recommendations for {title}:\n")
recommendationsD2V


Top 10 recommendations for Toy Story (1995):



,title,vote_count,vote_average,score,sentiment,cosine_similarity,sentiment_difference,combined_score
20962,Big Hero 6 (2014),10379,3,0.726134,0.357143,1.000000,0.328594,0.906895
574,Snow White and the Seven Dwarfs (1937),17940,3,0.485619,0.330000,0.936731,0.288507,0.846572
1890,"Little Mermaid, The (1989)",15145,3,0.438014,0.325000,0.806087,0.281122,0.751838
535,"Nightmare Before Christmas, The (1993)",21940,3,0.626667,-0.105000,0.770743,0.287388,0.744710
576,Pinocchio (1940),12742,3,0.391520,0.147222,0.688984,0.018559,0.717729
4007,Shrek (2001),42303,3,0.651150,0.155556,0.645083,0.030867,0.710500
961,Mary Poppins (1964),15121,3,0.607601,0.350000,0.693548,0.318045,0.682634
4876,Ice Age (2002),18215,3,0.448998,-0.100000,0.619823,0.280003,0.622775
14189,Despicable Me (2010),9658,3,0.567927,-0.494444,0.749887,0.862565,0.609200
2877,Toy Story 2 (1999),26536,3,0.694943,0.380000,0.584498,0.362352,0.606172


In [37]:
# Get recommendations for a specific movie
movie_id = 1
top_n = 10

recommendationsD2VOptimized = recommenderD2VOptimized.recommend(movie_id, top_n)

title = movies_df[movies_df['movieId'] == movie_id]['title'].to_string(index=False, header=False)
print(f"\nTop {top_n} recommendations for {title}:\n")
recommendationsD2VOptimized


Top 10 recommendations for Toy Story (1995):



,title,vote_count,vote_average,score,sentiment,cosine_similarity,sentiment_difference,combined_score
2877,Toy Story 2 (1999),26536,3,0.800259,0.380000,1.000000,0.362352,0.907555
576,Pinocchio (1940),12742,3,0.601587,0.147222,0.831707,0.018559,0.838642
1131,"Grand Day Out with Wallace and Gromit, A (1989)",7695,4,0.903041,0.475000,0.690006,0.502660,0.672776
1890,"Little Mermaid, The (1989)",15145,3,0.632030,0.325000,0.645715,0.281122,0.658979
4876,Ice Age (2002),18215,3,0.639222,-0.100000,0.602024,0.280003,0.629339
350,"Lion King, The (1994)",42745,3,0.808091,-0.250000,0.637173,0.501541,0.626522
574,Snow White and the Seven Dwarfs (1937),17940,3,0.663200,0.330000,0.589902,0.288507,0.621550
20962,Big Hero 6 (2014),10379,3,0.820682,0.357143,0.565615,0.328594,0.612279
4007,Shrek (2001),42303,3,0.771585,0.155556,0.473384,0.030867,0.602354
1815,"Goonies, The (1985)",11854,3,0.690259,-0.050000,0.497104,0.206157,0.575767


In [38]:
# Save the Doc2Vec recommender model
start_time = time.time()
with open('../02_Models/content_recommender_D2V.pkl', 'wb') as f:
    pickle.dump(recommenderD2V, f)
end_time = time.time()
doc2vec_save_time = end_time - start_time
print(f"Time taken to save Doc2Vec model: {doc2vec_save_time:.2f} seconds")

Time taken to save Doc2Vec model: 5.52 seconds


In [39]:
# Measure the size of the recommender object
size_in_bytes = asizeof.asizeof(recommenderD2V)
size_in_kb = size_in_bytes / 1024
size_in_mb = size_in_kb / 1024
print(f"The size of the recommender object is approximately {size_in_bytes} bytes, {size_in_kb:.2f} KB, or {size_in_mb:.2f} MB.")

The size of the recommender object is approximately 19285061864 bytes, 18833068.23 KB, or 18391.67 MB.


In [40]:
# Save the Doc2Vec recommender model
start_time = time.time()
with open('../02_Models/content_recommender_D2V_opt.pkl', 'wb') as f:
    pickle.dump(recommenderD2VOptimized, f)
end_time = time.time()
doc2vec_save_time = end_time - start_time
print(f"Time taken to save Doc2Vec Optimized model: {doc2vec_save_time:.2f} seconds")

Time taken to save Doc2Vec Optimized model: 0.57 seconds


In [41]:
# Measure the size of the recommender object
size_in_bytes = asizeof.asizeof(recommenderD2VOptimized)
size_in_kb = size_in_bytes / 1024
size_in_mb = size_in_kb / 1024
print(f"The size of the recommender object is approximately {size_in_bytes} bytes, {size_in_kb:.2f} KB, or {size_in_mb:.2f} MB.")

The size of the recommender object is approximately 158796160 bytes, 155074.38 KB, or 151.44 MB.


---

## 4.2 Modelling of Item-Based Collaborative-Filtering Recommender Systems

In collaborative filtering the recommender system purely learns form the interaction patterns between users and items. The contents and features of the items and users are completely ignored. Users and items are just treated as enumerated nodes of an undirected (weighted or unweighted) bipartite graph $G = (U \cup I, E)$ where the items I are indexed as $i_{k}$ and the User are indexed as $u_{j}$. Nothing more than the vertices, edges ${u_{j}i_{k}}$ and maybe a some edge weights ${w_{u_{j}i_{k}}}$ are known. Hence collaborative filtering corresponds to predicting promising links from user nodes to item nodes based on the observed common connection patterns. It is called collaborative filtering, because in the collaborative filtering approaches it is commonly assumed that learning the interaction patterns of one user (e.g., the items a user has interacted with) will help to predict relevant items for another user that has a similar interaction pattern (in terms of interacted items) as the latter user. Hence it is as if users were collaborating to produce the rankings of items for each other. 

Item-based collaborative filtering is often preferred over user-based collaborative filtering because it tends to perform better in cases where there are many items and relatively fewer users, which is often the case in recommendation systems.

 - One reason for this is that item-based collaborative filtering relies on the similarity between items to make recommendations, whereas user-based collaborative filtering relies on the similarity between users. It is often easier to measure the similarity between items than between users, especially when the number of items is large.

 - Another reason is that item-based collaborative filtering is more scalable than user-based collaborative filtering because the similarity matrix between items can be precomputed and reused, whereas the similarity matrix between users must be recomputed each time a new user is added to the system.

### **4.2.1 Loading and preparing data**

In [44]:
# load data
movies = pd.read_csv('../00_Data/01_processed/prepr_movies.csv', encoding='latin-1').iloc[:,1:]
ratings = pd.read_csv('../00_Data/01_processed/prepr_ratings.csv').iloc[:,1:]
ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

In [45]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25)

### **4.2.2 Training Item-Based Collaborative-Filtering Recommender Model with SVD**

We decided to implement Single Value Decomposition for the ratings based recommender for two main reasons:

Firstly, SVD is a suitable solution for handling the sparsity of the dataset. In real-world applications such as our ratings data, user-item rating matrices are often sparse, meaning that most entries are missing because users typically rate only a small fraction of the available items. SVD can effectively deal with this sparsity by identifying the latent factors that explain the observed ratings and using them to predict missing values. This makes SVD particularly suitable for ratings-based recommender systems.

In addition, SVD shows a good scalability, i.e., it can be applied to large-scale datasets, making it suitable for real-world ratings-based recommender systems. SVD helps in reducing the dimensionality of the user-item ratings matrix by decomposing it into three matrices - the user matrix, the singular value matrix, and the item matrix. This allows the system to capture the underlying structure and latent factors that drive user preferences, resulting in a more compact and efficient representation of the data.

To select the right number of components of the SVD, we conducted an experiment where we evaluated 3 different options: 50, 150, and 250 components – the first of which was the one with the best RMSE.

In [46]:
# Singular Value Decomposition (SVD)
svd = SVD(n_factors=50)
svd.fit(trainset)
predicts = svd.test(testset)

### **4.2.3 Building functions for data**

In [47]:
def get_top_n_similar_movies(movie_id, movies_df, svd, n=10):
    """
    Return the top N (default) most similar movies for a given movie_id

    Args:
        movie_id (int): Movie ID
        movies_df (pandas.DataFrame): DataFrame containing movies data
        svd (surprise.prediction_algorithms.matrix_factorization.SVD): Trained SVD model
        n (int, optional): Number of top similar movie recommendations. Defaults to 10.

    Returns:
        pandas.DataFrame: DataFrame containing top N similar movies
    """
    # Get the latent factors for all movies from the SVD model
    movie_factors = svd.qi
    movie_ids = {movie_id: index for index, movie_id in enumerate(svd.trainset._raw2inner_id_items)}

    # Calculate the similarity between the given movie and all other movies
    if movie_id in movie_ids:
        target_movie_index = movie_ids[movie_id]
        target_movie_factors = movie_factors[target_movie_index]

        similarities = np.dot(movie_factors, target_movie_factors)
        sorted_similarities = np.argsort(similarities)[::-1]

        # Create a DataFrame with similar movies and their similarity scores
        similar_movie_ids = [svd.trainset.to_raw_iid(index) for index in sorted_similarities[:n+1] if index != target_movie_index]
        similarity_scores = [similarities[index] for index in sorted_similarities[:n+1] if index != target_movie_index]

        similar_movies_df = pd.DataFrame({"movieId": similar_movie_ids, "similarity": similarity_scores})
        similar_movies_df = similar_movies_df.merge(movies_df, how='left', on='movieId')

        return similar_movies_df

    else:
        print(f"Movie ID {movie_id} not found in the training set.")
        return None

In [48]:
get_top_n_similar_movies(movie_id=1, movies_df=movies, svd=svd, n=10)

,movieId,similarity,title,genres,year,tmdbId,tag,collection_name,original_language,description,...,description_meanword_wsw,description_nchars,description_nchars_wsw,description_diff_nchars,description_root_wrds,description_jj_n,description_nn_n,description_prp_n,description_rb_n,description_vb_n
0,3114,3.047025,Toy Story 2 (1999),"['Adventure', 'Animation', 'Children', 'Comedy...",1999.0,863.0,['2009reissueinstereoscopic3-d' '3d' 'abandonm...,Toy Story Collection,en,"Andy heads off to Cowboy Camp, leaving his toy...",...,5.289474,318.0,238.0,80.0,andy head cowboy camp leaving toy device thing...,0.0,28.0,7.0,2.0,6.0
1,78499,3.007004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,595,2.752683,Beauty and the Beast (1991),"['Animation', 'Children', 'Fantasy', 'Musical'...",1991.0,10020.0,['18thcentury' '2danimation'\n '55movieseveryk...,Beauty and the Beast Collection,en,"Follow the adventures of Belle, a bright young...",...,5.846154,272.0,177.0,95.0,follow adventure belle bright young woman find...,1.0,18.0,2.0,2.0,7.0
3,588,2.566989,Aladdin (1992),"['Adventure', 'Animation', 'Children', 'Comedy...",1992.0,812.0,['(s)vcd' '2danimation' 'action' 'adventure' '...,Aladdin Collection,en,Princess Jasmine grows tired of being forced t...,...,6.114286,372.0,248.0,124.0,princess jasmine grows tired forced remain pal...,3.0,29.0,4.0,7.0,8.0
4,34,2.531504,Babe (1995),"['Children', 'Drama']",1995.0,9598.0,['55movieseverykidshouldsee--entertainmentweek...,Babe Collection,en,Babe is a little pig who doesn't quite know hi...,...,5.000000,382.0,233.0,149.0,babe little pig n't quite know place world bun...,2.0,32.0,6.0,2.0,8.0
5,6377,2.379241,Finding Nemo (2003),"['Adventure', 'Animation', 'Children', 'Comedy']",2003.0,12.0,['55movieseverykidshouldsee--entertainmentweek...,Finding Nemo Collection,en,"Nemo, an adventurous young clownfish, is unexp...",...,6.305556,329.0,262.0,67.0,nemo adventurous young clownfish unexpectedly ...,2.0,34.0,7.0,2.0,8.0
6,4886,2.350005,"Monsters, Inc. (2001)","['Adventure', 'Animation', 'Children', 'Comedy...",2001.0,585.0,['3' 'andrewstanton' 'animated' 'animation' 'b...,"Monsters, Inc. Collection",en,"James Sullivan and Mike Wazowski are monsters,...",...,6.117647,383.0,241.0,142.0,james sullivan mike wazowski monster earn livi...,3.0,21.0,4.0,2.0,6.0
7,2081,2.240837,"Little Mermaid, The (1989)","['Animation', 'Children', 'Comedy', 'Musical',...",1989.0,10144.0,['2danimation' '55movieseverykidshouldsee--ent...,The Little Mermaid Collection,en,This colorful adventure tells the story of an ...,...,6.222222,274.0,194.0,80.0,colorful adventure tell story impetuous mermai...,0.0,9.0,2.0,1.0,3.0
8,364,2.237621,"Lion King, The (1994)","['Adventure', 'Animation', 'Children', 'Drama'...",1994.0,8587.0,['2danimation' '55movieseverykidshouldsee--ent...,The Lion King Collection,en,A young lion prince is cast out of his pride b...,...,5.351351,394.0,234.0,160.0,young lion prince cast pride cruel uncle claim...,5.0,23.0,4.0,3.0,8.0
9,8961,2.222428,"Incredibles, The (2004)","['Action', 'Adventure', 'Animation', 'Children...",2004.0,9806.0,"[""'60sfeel"" '007-like' '1.5'\n '55movieseveryk...",The Incredibles Collection,en,Bob Parr has given up his superhero days to lo...,...,6.043478,232.0,161.0,71.0,bob parr given superhero day log time insuranc...,1.0,11.0,0.0,1.0,1.0


In [49]:
# Function to retrieve top n recommendations for a given user
def get_top_n(predictions, user_id, movies_df, ratings_df, n=10):
    """
    Return the top N (default) movieId for a user, i.e. userID and history for comparison

    Args:
        predictions (list): List of tuples (uid, iid, true_r, est, _)
        user_id (int): User ID
        movies_df (pandas.DataFrame): DataFrame containing movies data
        ratings_df (pandas.DataFrame): DataFrame containing ratings data
        n (int, optional): Number of top movie recommendations. Defaults to 10.

    Returns:
        tuple: Two DataFrames - hist_usr and pred_usr
    """    
    # 1. First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # 2. Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    # 3. Tells how many movies the user has already rated
    user_data = ratings_df[ratings_df.user_id == user_id]
    print(f"User {user_id} has already rated {user_data.shape[0]} movies.")

    # 4. Data Frame with predictions.
    preds_df = pd.DataFrame([(id, pair[0], pair[1]) for id, row in top_n.items() for pair in row], columns=["userId", "movieId", "rat_pred"])

    # 5. Return pred_usr, i.e. top N recommended movies with (merged) titles and genres.
    pred_usr = preds_df[preds_df["userId"] == user_id].merge(movies_df, how='left', left_on='movieId', right_on='movieId')

    # 6. Return hist_usr, i.e. top N historically rated movies with (merged) titles and genres for holistic evaluation
    hist_usr = ratings_df[ratings_df.user_id == user_id].sort_values("rating", ascending=False).merge(movies_df, how='left', left_on='movie_id', right_on='movieId')

    return hist_usr, pred_usr

### **4.2.4 Testing**

In [50]:
hist_SVD_124, pred_SVD_124 = get_top_n(predicts, movies_df = movies, user_id = 124, ratings_df = ratings)

User 124 has already rated 55 movies.


In [51]:
hist_SVD_124

,user_id,movie_id,rating,timestamp,movieId,title,genres,year,tmdbId,tag,...,description_meanword_wsw,description_nchars,description_nchars_wsw,description_diff_nchars,description_root_wrds,description_jj_n,description_nn_n,description_prp_n,description_rb_n,description_vb_n
0,124,111,5.0,833210442,111.0,Taxi Driver (1976),"['Crime', 'Drama', 'Thriller']",1976.0,103.0,['5stars' 'acting' 'afi#47' 'afi100' 'afi100(m...,...,6.000000,165.0,132.0,33.0,mentally unstable vietnam war veteran work nig...,0.0,5.0,2.0,1.0,3.0
1,124,1183,4.0,852303305,1183.0,"English Patient, The (1996)","['Drama', 'Romance', 'War']",1996.0,409.0,['adaptedfrom:book' 'adultery' 'africa' 'airpl...,...,6.137931,273.0,206.0,67.0,1930s count almÃ¡sy hungarian map maker employ...,5.0,34.0,6.0,3.0,2.0
2,124,194,4.0,833211978,194.0,Smoke (1995),"['Comedy', 'Drama']",1995.0,10149.0,['brooklyn' 'cigar' 'cigarette' 'ensemblecast'...,...,5.704545,399.0,294.0,105.0,writer paul benjamin nearly hit bus leaf auggi...,1.0,9.0,4.0,1.0,5.0
3,124,608,4.0,852303272,608.0,Fargo (1996),"['Comedy', 'Crime', 'Drama', 'Thriller']",1996.0,275.0,"['""ohyah""' '1980s' '3' 'absurd' 'accent' 'acti...",...,5.947368,545.0,395.0,150.0,jerry small-town minnesota car salesman bursti...,6.0,42.0,14.0,7.0,12.0
4,124,593,4.0,833210442,593.0,"Silence of the Lambs, The (1991)","['Crime', 'Horror', 'Thriller']",1991.0,274.0,['100essentialfemaleperformances' '1990s' '2.5...,...,6.317073,400.0,299.0,101.0,clarice starling top student fbi training acad...,2.0,6.0,0.0,0.0,1.0
5,124,527,4.0,833212136,527.0,Schindler's List (1993),"['Drama', 'War']",1993.0,424.0,['1930s' '2ndworldwar' '8.7-filmaffinity' 'abo...,...,5.750000,163.0,107.0,56.0,true story businessman oskar schindler saved t...,6.0,28.0,1.0,0.0,8.0
6,124,515,4.0,833210686,515.0,"Remains of the Day, The (1993)","['Drama', 'Romance']",1993.0,1245.0,['70mm' '70mmblowup' 'adaptedfrom:book' 'antho...,...,6.964286,311.0,222.0,89.0,rule bound head butler world manner decorum ho...,3.0,32.0,8.0,6.0,6.0
7,124,509,4.0,833210664,509.0,"Piano, The (1993)","['Drama', 'Romance']",1993.0,713.0,['100essentialfemaleperformances' '19thcentury...,...,6.431818,438.0,326.0,112.0,long voyage scotland pianist ada mcgrath young...,0.0,34.0,5.0,4.0,10.0
8,124,500,4.0,833212236,500.0,Mrs. Doubtfire (1993),"['Comedy', 'Drama']",1993.0,788.0,['afi100(laughs)' 'children' 'chriscolumbus' '...,...,5.970588,356.0,236.0,120.0,loving irresponsible dad daniel hillard estran...,7.0,28.0,6.0,3.0,5.0
9,124,471,4.0,833211931,471.0,"Hudsucker Proxy, The (1994)",['Comedy'],1994.0,11934.0,['1950s' 'bd-r' 'board' 'boardroomjungle' 'bos...,...,7.200000,103.0,81.0,22.0,naive business graduate installed president ma...,2.0,9.0,2.0,0.0,2.0


In [52]:
pred_SVD_124

,userId,movieId,rat_pred,title,genres,year,tmdbId,tag,collection_name,original_language,...,description_meanword_wsw,description_nchars,description_nchars_wsw,description_diff_nchars,description_root_wrds,description_jj_n,description_nn_n,description_prp_n,description_rb_n,description_vb_n
0,124,296,4.002117,Pulp Fiction (1994),"['Comedy', 'Crime', 'Drama', 'Thriller']",1994.0,680.0,['1990s' '90s' 'accidentalkilling' 'achronolog...,NaN,en,...,6.916667,237.0,189.0,48.0,burger-loving hit man philosophical partner dr...,1.0,13.0,0.0,0.0,0.0
1,124,307,3.823300,Three Colors: Blue (Trois couleurs: Bleu) (1993),['Drama'],1993.0,108.0,['2.5' 'atmospheric' 'bd-r' 'beauty' 'bestofro...,Three Colors Collection,fr,...,6.512195,464.0,307.0,157.0,julie haunted grief living tragic auto wreck c...,0.0,13.0,0.0,0.0,1.0
2,124,194,3.762452,Smoke (1995),"['Comedy', 'Drama']",1995.0,10149.0,['brooklyn' 'cigar' 'cigarette' 'ensemblecast'...,Brooklyn Cigar Store Collection,en,...,5.704545,399.0,294.0,105.0,writer paul benjamin nearly hit bus leaf auggi...,1.0,9.0,4.0,1.0,5.0
3,124,319,3.664700,Shallow Grave (1994),"['Comedy', 'Drama', 'Thriller']",1994.0,9905.0,['3' 'atmospheric' 'biting' 'blackcomedy' 'chr...,NaN,en,...,5.883721,420.0,295.0,125.0,accountant david doctor juliet journalist alex...,2.0,19.0,2.0,1.0,4.0
4,124,32,3.622954,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"['Mystery', 'Sci-Fi', 'Thriller']",1995.0,63.0,['3' 'absurd' 'adaptedfrom/inspiredby:shortfil...,NaN,en,...,6.258621,586.0,420.0,166.0,year 2035 convict james cole reluctantly volun...,3.0,38.0,2.0,10.0,8.0
5,124,121,3.567030,"Boys of St. Vincent, The (1992)",['Drama'],1992.0,32119.0,['1970s' 'basedonatruestory' 'catholicism' 'ch...,The Boys of St. Vincent,en,...,6.900000,110.0,78.0,32.0,true story boy sexually abused orphanage run r...,3.0,12.0,0.0,0.0,0.0
6,124,6,3.517022,Heat (1995),"['Action', 'Crime', 'Thriller']",1995.0,949.0,['1' '7.5-filmaffinity' 'action' 'adultery' 'a...,Heat Collection,en,...,6.529412,324.0,255.0,69.0,obsessive master thief neil mccauley lead top-...,4.0,24.0,3.0,1.0,3.0
7,124,474,3.430726,In the Line of Fire (1993),"['Action', 'Thriller']",1993.0,9386.0,"['70mm' 'action' 'action,aging' 'anamorphicblo...",NaN,en,...,6.314286,326.0,255.0,71.0,veteran secret service agent frank horrigan ma...,5.0,37.0,5.0,0.0,7.0
8,124,198,3.271851,Strange Days (1995),"['Action', 'Crime', 'Drama', 'Mystery', 'Sci-F...",1995.0,281.0,['90sdystopia' 'alcoholconsume' 'angelabassett...,NaN,en,...,6.062500,321.0,225.0,96.0,last day 1999 ex-cop turned street hustler len...,2.0,17.0,2.0,2.0,6.0
9,124,500,3.190852,Mrs. Doubtfire (1993),"['Comedy', 'Drama']",1993.0,788.0,['afi100(laughs)' 'children' 'chriscolumbus' '...,NaN,en,...,5.970588,356.0,236.0,120.0,loving irresponsible dad daniel hillard estran...,7.0,28.0,6.0,3.0,5.0


---

## 4.3 Modelling of Hybrid Recommender Systems

A hybrid recommender system synergistically combines two distinct recommendation strategies, namely Singular Value Decomposition (SVD) for item-based collaborative filtering (Koren et al., 2009), and content-based recommendation (Aggarwal, 2016). This approach aims to capitalize on the strengths of both methods to provide more accurate and relevant recommendations for users.

The SVD decomposition component of the hybrid model employs the SVD method from the Surprise library to train on user-item interactions, learning latent components in the process (Salakhutdinov & Mnih, 2008). The model selects the top n items with the highest expected ratings for a given user to generate personalized recommendations. The get_top_n() function takes the SVD model predictions as input and returns a list of suggested items for the specified user.

In contrast, the content-based recommendation component of the hybrid model utilizes an optimized Term Frequency-Inverse Document Frequency (TF-IDF) Vectorizer to generate recommendations based on movie metadata, such as actors, directors, tags, and descriptions (Aggarwal, 2016). Cosine similarity serves as the metric to identify the most similar movies. The recommend() function accepts a movie ID as input and returns the top n recommendations that bear resemblance to the provided movie.

The hybrid model integrates the results of both components to generate a comprehensive set of user recommendations. The hybrid_recommender() function initially produces content-based recommendations using the content-based recommender model (Aggarwal, 2016). Subsequently, it employs the SVD model to estimate ratings for the top 100 content-based movies (Salakhutdinov & Mnih, 2008). These estimated ratings are sorted in descending order, and the top n recommendations are selected.

In summary, the hybrid recommender system consolidates the advantages of two distinct recommendation techniques, while harnessing the power of the TF-IDF Vectorizer to discern semantic relationships and contextual information within the movie metadata (Aggarwal, 2016). This approach is expected to yield more precise and pertinent recommendations for users.

### **4.3.1 Importing Content-Based and Item-Based Collaborative Filtering Models**

**Load content-based recommender model (TF-IDF Vectorizer Optimized)**

In [53]:
# Load the saved model
with open('../02_Models/content_recommender_tfidf.pkl', 'rb') as file:
    recommender = pickle.load(file)

**Load item-based collaborative-filtering recommender model (SVD)**

In [54]:
# Start tracking time
start_time = time.time()

# Create a reader with a rating scale from 1 to 5
reader = Reader(rating_scale=(1, 5))

# Load the ratings data into a Surprise Dataset format
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

# Build a full trainset for the SVD model
full_trainset = data.build_full_trainset()

# Function to load the SVD model
def load_model(model_path):
    return joblib.load(model_path)

# Load the pre-trained SVD model
svd = load_model('../02_Models/svd_model.pkl')

# Fit the SVD model on the full trainset
svd.fit(full_trainset)

# Calculate and print the elapsed time
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 338.79 seconds


### **4.3.2 Bulding hybrid model**

In [55]:
def hybrid_recommender(user_id, movie_id, movies_df, ratings_df, top_n=10):
    # 1. Get content-based recommendations for top 100 movies
    content_recommendations = recommender.recommend(movie_id, top_n=100)
    content_recommendations.reset_index(inplace = True)
    content_recommendations.rename(columns = {"index": "movieId"}, inplace = True)

    # 2. Use SVD model to estimate ratings for the top 100 content-based movies
    top_ratings = []
    for index, row in content_recommendations.iterrows():
        if row['movieId'] != movie_id:
          est_rating = svd.predict(user_id, row['movieId']).est
          top_ratings.append((row['movieId'], row['title'], est_rating))

    movie_recommendations = pd.DataFrame(top_ratings, columns=['movie_id', 'title', 'est_rating'])
    movie_recommendations = movie_recommendations.sort_values('est_rating', ascending=False)

    # 3. Select the top n recommendations
    movie_recommendations = movie_recommendations.head(top_n)

    return movie_recommendations

### **4.3.3 Testing hybrid model**

In [56]:
user_id = 124
movie_id = 1

# Running hybrid recommendation for user 124 and movie 1
hybrid_recommendations = hybrid_recommender(user_id, movie_id, movies_df, ratings)
hybrid_recommendations

,movie_id,title,est_rating
39,593,"Aristocats, The (1970)",4.097001
7,4007,Shrek (2001),3.805272
8,1176,Back to the Future (1985),3.707239
3,7884,"Incredibles, The (2004)",3.691162
24,249,Star Wars: Episode IV - A New Hope (1977),3.657568
29,962,Dumbo (1941),3.566987
23,1171,Groundhog Day (1993),3.547778
32,10863,Ratatouille (2007),3.534142
22,13769,How to Train Your Dragon (2010),3.534142
1,14080,Toy Story 3 (2010),3.534142


---

## **Sources**

    Aggarwal, C. C. (2016). Content-based recommender systems. In Recommender Systems (pp. 139-166). Springer, Cham.
    
    Han, J., Kamber, M., & Pei, J. (2011). Data mining: concepts and techniques. Elsevier.

    Hugging Face. (n.d.). Hugging Face - On a mission to solve NLP, one commit at a time.

    Huang, A. (2008). Similarity measures for text document clustering. Proceedings of the sixth New Zealand Computer Science Research Student Conference.
    
    Koren, Y., Bell, R., & Volinsky, C. (2009). Matrix factorization techniques for recommender systems. Computer, 42(8), 30-37.

    Manning, C. D., Raghavan, P., & Schütze, H. (2008). Introduction to information retrieval. Cambridge University Press.

    Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space.

    Omohundro, S. M. (1989). Five balltree construction algorithms. International Computer Science Institute, Berkeley, 89(2).
    
    Pazzani, M. J., & Billsus, D. (2007). Content-based recommendation systems. In P. Brusilovsky, A. Kobsa, & W. Nejdl (Eds.), The Adaptive Web: Methods and Strategies of Web Personalization (pp. 325-341). Springer Berlin Heidelberg.

    Pennington, J., Socher, R., & Manning, C. (2014). GloVe: Global Vectors for Word Representation. Proceedings of the 2014 Conference on Empirical Methods in Natural Language Processing (EMNLP), 1532–1543.
    
    Ricci, F., Rokach, L., & Shapira, B. (2011). Introduction to Recommender Systems Handbook. In F. Ricci, L. Rokach, B. Shapira, & P. B. Kantor (Eds.), Recommender Systems Handbook (pp. 1-35). Springer US.
    
    Salakhutdinov, R., & Mnih, A. (2008). Probabilistic matrix factorization. In Advances in neural information processing systems (pp. 1257-1264).

    Sanh, V., Debut, L., Chaumond, J., & Wolf, T. (2019). DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter.

    Sarwar, B., Karypis, G., Konstan, J., & Riedl, J. (2001). Item-based collaborative filtering recommendation algorithms. Proceedings of the 10th International Conference on World Wide Web (WWW '01), 285-295.

    Song, K., Tan, X., Qin, T., Lu, J., & Liu, T. Y. (2020). MPNet: Masked and Permuted Pre-training for Language Understanding.

    Sun, C., Qiu, X., & Huang, X. (2019). Utilizing BERT for Aspect-Based Sentiment Analysis via Constructing Auxiliary Sentence.